In [1]:
from video_process_pck.motion_detector import *
from video_process_pck.plot_mvm import plot_mvm
import os
import numpy as np
import matplotlib.pyplot as plt

## **calculate big differences:**
+ index
+ length
+ rate

In [12]:
#path to SYNC_file
ephys_time_path="/media/data-119/rat596_20210702_175129/rat596_20210702_175129tmstp.txt"

#read SYNC_file
e_test = open(ephys_time_path,"r")
ephys_time_test = [[int(x) for x in line.split()] for line in e_test]
e_test.close()

#concatenate the list of lists into one list (array)
ephys_time_test = np.concatenate(ephys_time_test)

#convert to milliseconds 
ephys_time_test_ms=[i/2.5 for i in ephys_time_test]

#get the difference between frame in milliseconds
ephys_time_test_ms_diff=np.diff(ephys_time_test_ms)

#get index lags.. time tn-t0
differences_ms=[]
for i in range(len(ephys_time_test_ms_diff)):
    
    #if the diffed time is not exactly 40 ms
    if ephys_time_test_ms_diff[i]-40 != 0:
        #can be negative or positive difference
        differences_ms.append((i,ephys_time_test_ms_diff[i]-40))
        
#big differences -> from differences select diff of absolute value bigger than 0.0001ms
big_differences_ms=[]
for i in range(len(differences_ms)):
    
    #append tuple if np.abs(difference) > 0.0001
    if np.abs(differences_ms[i][1]) > 0.0001:
        big_differences_ms.append(differences_ms[i])
        
#get the indexes only of big diffs
big_differences_ms_indexes=[big_differences_ms[i][0] for i in range(len(big_differences_ms))]

#np diff of indexes -> after how many frames the difference occur
big_differences_ms_indexes_DIFFS=np.diff(big_differences_ms_indexes)

## **Account for lags -> delete each 3850th frame -> when the sum of lags adds up to 40 ms**

### Take time from ephys file in your df

In [64]:
#try: cap_prop_pos_msec... https://www.quora.com/How-do-I-decrease-the-frames-per-second-in-OpenCV-python

def motion_detector_3850th(path, scale_percent=40, area=20, delta_thresh=1,
                    output_4csv="/home/domi/video_proc/TESTS/md3850/",
                   ephys_time_path="/media/data-119/rat596_20210702_175129/rat596_20210702_175129tmstp.txt"):
    """
    To account for lags in ephys timestamps: skip every 3850th frame (that's number of frames after which the lag
    adds up to 40ms => which is duration of one standard frame
    
    path: path to the video
    area: minimum area size.... TO INCLUDE!!!
    ephys_time: path to ephys timestamp.. load it and convert to millisec (divide by 2.5) 
        -> "duration" for df
    """
    
    # 1) EPHYS
    #read file
    e = open(ephys_time_path,"r")
    ephys_time = [[int(x) for x in line.split()] for line in e]
    e.close()
    ephys_time = np.concatenate(ephys_time)
    
    #convert eph to millisec
    ephys_time = ephys_time/2.5           
                                         
        
    #INSERT CODE TO VERIFY THAT THE RATE OF LAGS IS CTE
    #------------------------------------------------------------------------------
    #------------------------------------------------------------------------------
    
    #to calculate the duration of processed video the timestamp have to start at 0
    ephys_time_0 = [i - ephys_time[0] for i in ephys_time]
    
    
    # 2) OUTPUT
    #if specified output folder doesn't exist, create it
    if not os.path.exists(output_4csv):
        os.mkdir(output_4csv)


    # 3) VID_PROC
    # read from a video file 
    vs = cv2.VideoCapture(path)
    
    #set frame rate to 25
    print(vs.get(cv2.CAP_PROP_FPS))
    vs.set(cv2.CAP_PROP_FPS, 25)
    print(vs.get(cv2.CAP_PROP_FPS))
    
    #initiate FramePerSec (FPS) count
    fps = FPS().start()

    # initialize average frame, frame delta and thresholded frame
    avgframe = [[], []]
    frameDelta = [[], []]
    thresh = [[], []]

    # initiate dictionary
    ts_dict = {"frame_nb": [], "millisec": [], "mvm_rat1": [], "mvm_rat2": []}

    # initiate dimensions for splitting
    # n_rows=1
    n_cols = 2  # 2 rats = 2 parts split with a vertical line
                # division in the middle

    # loop over the frames of the video
    n = 0
    
    #DELETE AFTER___________________________________________________________________________________________
    #milliseconds = []
    frame_exists=[]
    #eph_time=[]
    #________________________________________________________________________________________________________
    
    while True:
        
        ret, frame = vs.read()  # ret says whether the frame exists
        frame_exists.append(ret)
        
        if frame is None:
            break  # end of video if no more frames

        # get timestamp of each frame...
        frame_nb = n
        
        #IF (FRAME NUMBER-21) MODULO 3850 == 0, THEN SKIP THIS FRAME ..... -> except 0 (0 included -> 21st frame)
        if (n-big_differences_ms_indexes[0])%3850 == 0:
            
            #itter n
            n += 1
            
            #skip the frame
            continue
            
            
        ########################    
        #print(n)
        ######################
        
        millisec = vs.get(cv2.CAP_PROP_POS_MSEC)  # in millisecond
        
        #DELETE AFTER___________________________________________________________________________________________
        #milliseconds.append(vs.get(cv2.CAP_PROP_POS_MSEC))
        
        #for duration -> it will be NOT precise.. but difference under 40ms
        #eph_time.append(ephys_time_0[n])
        #_________________________________________________________________________________________________________
        
        ts_dict = timestamping(ts_dict=ts_dict,
                               frame_nb=frame_nb,
                               millisec=millisec)
        
        # resize the frame, convert it to grayscale, and blur it
        frame = resizing(frame=frame, scale_percent=scale_percent)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        # get gray's shape for splitting............................................
        height, width = gray.shape

        # divide the frame and calculate background and change for each part
        for i in range(n_cols):
            tmp_img = gray[0:height, int(i * width / 2): int((i + 1) * width / 2)]  # take all hight and half of width

            # initialize average frame (=background)
            if len(avgframe[i]) == 0:
                avgframe[i] = tmp_img.copy().astype("float")

            # update average & calculate difference between current and running avg
            cv2.accumulateWeighted(tmp_img, avgframe[i], 0.5)
            frameDelta[i] = cv2.absdiff(tmp_img,
                                        cv2.convertScaleAbs(avgframe[i]))

            # threshold the delta image, dilate the thresholded image to fill
            # in holes, then find contours on thresholded image
            thresh[i] = cv2.threshold(frameDelta[i], delta_thresh, 255,
                                      cv2.THRESH_BINARY)[1]
            thresh[i] = cv2.dilate(thresh[i], None, iterations=1)  # spread the white pixels

        # append mvm count for each rat
        for i in range(2):
            ts_dict[f"mvm_rat{i + 1}"].append(np.sum(thresh[i]))
        
        #itter loop
        n += 1
        
        # update FPS counter
        fps.update()

        # DISPLAY VIDEOS (IT'S 2x LONGER THIS WAY... 92:177 FPS)!!!
        # cv2.imshow("webcam", frame)
        # cv2.imshow("Thresh_rat1", thresh[0])
        # cv2.imshow("Thresh_rat2", thresh[1])
        # cv2.imshow("Frame Delta_rat2", frameDelta[1])

        # cv2.waitKey(1)
        keyboard = cv2.waitKey(1)
        #if keyboard == 'q' or keyboard == 27:
        #    break

        # testing, remove after...................................................................................
        #if n > 25 * 60 * 6: #5 minutes
        
    cv2.destroyAllWindows()
    newpath, first_stamp = csv_name_creator(path, output_folder=output_4csv)
    ts_df = pd.DataFrame(ts_dict)

    #instead of video-itter-count use ephys timestamp for 'duration'
    ts_df['duration_eph'] = ephys_time_0[n]
    ts_df['duration_eph'] = ts_df['duration_eph'].values.astype('datetime64[ns]')
    ts_df['duration_eph'] = [i.strftime("%H:%M:%S.%f")[:-3] for i in ts_df['duration_eph']]


    ts_df['duration'] = [timedelta(milliseconds=float(i)) for i in ts_df['millisec']]
    ts_df['duration'] = ts_df['duration'].values.astype('datetime64[ns]')
    ts_df['duration'] = [i.strftime("%H:%M:%S.%f")[:-3] for i in ts_df['duration']]

    ts_df['timestamp'] = [pd.to_timedelta(str(i) + 'millisecond') +
                                 first_stamp for i in ts_df['millisec']]
    ts_df.set_index('timestamp', inplace=True)

    order = [0, 1, 4, 5, 2, 3]  # setting column's order
    ts_df = ts_df[[ts_df.columns[i] for i in order]]

    ts_df.to_csv(csv_name_creator(path, output_folder=output_4csv)[0], sep='\t')

    # stop the timer and display FPS information
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    #save df as csv
    ts_df.to_csv(newpath, sep='\t')

    # return the df with timestamp+mvm count
    return ts_df , frame_exists#..........................................................................................



    # stop the timer and print Frame Per Sec info
    fps.stop()
    print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    # CLEAN UP
    cv2.destroyAllWindows()

    # get path for csv output & init of timestamp (extracted from the video filename)
    newpath, first_stamp = csv_name_creator(path, output_folder=output_4csv)

    # convert dict do pandas data frame...
    ts_df = pd.DataFrame(ts_dict)

    # create column 'duration' (millisec converted to time)
    ts_df['duration'] = [timedelta(milliseconds=float(i)) for i in ts_df['millisec']]
    # to keep only the time of the timedelta (otherwise give you estimate with days as well)
    ts_df['duration'] = ts_df['duration'].values.astype('datetime64[ns]')
    ts_df['duration'] = [i.strftime("%H:%M:%S.%f")[:-3] for i in ts_df['duration']]

    # ...set exact datetime timestamp (adding the first stamp to milliseconds)...
    ts_df['timestamp'] = [pd.to_timedelta(str(i) + 'millisecond') +
                          first_stamp for i in ts_df['millisec']]
    ts_df.set_index('timestamp', inplace=True)

    order = [0, 1, 4, 2, 3]  # setting columns' order
    ts_df = ts_df[[ts_df.columns[i] for i in order]]

    # ...and save it to info directory
    ts_df.to_csv(newpath, sep='\t')
    # return ts_df

In [79]:
df_3850, f_exists = motion_detector_3850th("/media/data-108/Videos/video_W596_W597_W602/Basler_acA1300-60gmNIR__21471690__20210702_175129792.mp4",
                                          scale_percent=40, area=20, delta_thresh=1,
                    output_4csv="/home/domi/video_proc/TESTS/",
                   ephys_time_path="/media/data-119/rat596_20210702_175129/rat596_20210702_175129tmstp.txt")

25.0
25.0


IndexError: list index out of range

In [66]:
len(df_3850)

289314

In [67]:
len(ephys_time_test_ms)

1428413

https://stackoverflow.com/questions/35912335/how-to-extract-a-fixed-number-of-frames-with-ffmpeg

You can get the total number of frames, divide it by n to have the number of frames you'll skip at each step and then read the frames

vidcap = cv2.VideoCapture(video_path)
    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    frames_step = total_frames//n
    for i in range(n):
        #here, we set the parameter 1 which is the frame number to the frame (i*frames_step)
        vidcap.set(1,i*frames_step)
        success,image = vidcap.read()  
        #save your image
        cv2.imwrite(path,image)
    vidcap.release()

In [33]:
ms_3850

[0.0,
 41.666666666666664,
 83.33333333333333,
 125.0,
 166.66666666666666,
 208.33333333333331,
 250.0,
 291.66666666666663,
 333.3333333333333,
 375.0,
 416.66666666666663,
 458.3333333333333,
 500.0,
 541.6666666666666,
 583.3333333333333,
 625.0,
 666.6666666666666,
 708.3333333333333,
 750.0,
 791.6666666666666,
 833.3333333333333,
 916.6666666666666,
 958.3333333333333,
 1000.0,
 1041.6666666666665,
 1083.3333333333333,
 1125.0,
 1166.6666666666665,
 1208.3333333333333,
 1250.0,
 1291.6666666666665,
 1333.3333333333333,
 1375.0,
 1416.6666666666665,
 1458.3333333333333,
 1500.0,
 1541.6666666666665,
 1583.3333333333333,
 1625.0,
 1666.6666666666665,
 1708.3333333333333,
 1750.0,
 1791.6666666666665,
 1833.3333333333333,
 1875.0,
 1916.6666666666665,
 1958.3333333333333,
 2000.0,
 2041.6666666666665,
 2083.333333333333,
 2125.0,
 2166.6666666666665,
 2208.333333333333,
 2250.0,
 2291.6666666666665,
 2333.333333333333,
 2375.0,
 2416.6666666666665,
 2458.333333333333,
 2500.0,
 254

In [28]:
df_3850.tail()

,frame_nb,millisec,duration_eph,duration,mvm_rat1,mvm_rat2
timestamp,,,,,,
2021-06-16 16:32:01.007,289385,0.0,00:00:00.011,00:00:00.000,0,0
2021-06-16 16:32:01.007,289386,0.0,00:00:00.011,00:00:00.000,0,0
2021-06-16 16:32:01.007,289387,0.0,00:00:00.011,00:00:00.000,0,0
2021-06-16 16:32:01.007,289388,0.0,00:00:00.011,00:00:00.000,0,0
2021-06-16 16:32:01.007,289389,0.0,00:00:00.011,00:00:00.000,0,0


In [29]:
ms_3850

[0.0,
 41.666666666666664,
 83.33333333333333,
 125.0,
 166.66666666666666,
 208.33333333333331,
 250.0,
 291.66666666666663,
 333.3333333333333,
 375.0,
 416.66666666666663,
 458.3333333333333,
 500.0,
 541.6666666666666,
 583.3333333333333,
 625.0,
 666.6666666666666,
 708.3333333333333,
 750.0,
 791.6666666666666,
 833.3333333333333,
 916.6666666666666,
 958.3333333333333,
 1000.0,
 1041.6666666666665,
 1083.3333333333333,
 1125.0,
 1166.6666666666665,
 1208.3333333333333,
 1250.0,
 1291.6666666666665,
 1333.3333333333333,
 1375.0,
 1416.6666666666665,
 1458.3333333333333,
 1500.0,
 1541.6666666666665,
 1583.3333333333333,
 1625.0,
 1666.6666666666665,
 1708.3333333333333,
 1750.0,
 1791.6666666666665,
 1833.3333333333333,
 1875.0,
 1916.6666666666665,
 1958.3333333333333,
 2000.0,
 2041.6666666666665,
 2083.333333333333,
 2125.0,
 2166.6666666666665,
 2208.333333333333,
 2250.0,
 2291.6666666666665,
 2333.333333333333,
 2375.0,
 2416.6666666666665,
 2458.333333333333,
 2500.0,
 254